<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/Tyre_Quality_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install opendatasets library
!pip install opendatasets

In [2]:
# Import necessary libraries and functions
import pandas as pd
import numpy as np
import opendatasets as od
import os
import shutil
from keras.metrics import Precision
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
# Download the data into the working directory
od.download("https://www.kaggle.com/datasets/warcoder/tyre-quality-classification")

In [4]:
# Set some directories
classes = ['good', 'defective']
train_directory = "/content/train"
test_directory = "/content/test"
root_directory = "/content/tyre-quality-classification/Digital images of defective and good condition tyres"

# Iterate over classes and
for class_name in classes:
    class_directory = os.path.join(root_directory, class_name)
    images = os.listdir(class_directory) # List all images in the targeted path

    # Split the images into training and testing sets
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    # Create destination directories
    train_class_dir = os.path.join(train_directory, class_name)
    os.makedirs(train_class_dir, exist_ok=True)

    test_class_dir = os.path.join(test_directory, class_name)
    os.makedirs(test_class_dir, exist_ok=True)

    # Copy training images
    for image in train_images:
        source_path = os.path.join(class_directory, image)
        destination_path = os.path.join(train_class_dir, image)
        shutil.copy(source_path, destination_path)

    # Copy testing images
    for image in test_images:
        source_path = os.path.join(class_directory, image)
        destination_path = os.path.join(test_class_dir, image)
        shutil.copy(source_path, destination_path)


In [5]:
# Set the ImageDataGenerator for data augmentation
datagen=ImageDataGenerator(rescale=1.0/255.0,
      rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest')

train_generator=datagen.flow_from_directory("/content/train",class_mode="binary",batch_size=64,target_size=(224,224))
test_generator=datagen.flow_from_directory("/content/test",class_mode="binary",batch_size=64,target_size=(224,224))

Found 1484 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


In [6]:
# Define the mini VGG model
def mini_vgg():
    model = Sequential()

    # Block 1
    model.add(Conv2D(32,5, activation='relu', input_shape=(224,224,3)))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(32,3, activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Block 2
    model.add(Conv2D(64,4, activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Flatten
    model.add(Flatten())

    # Dense layers
    model.add(Dense(125, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [ ]:
# Compile and train the model
model=mini_vgg()
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy",Precision()])
model.fit(train_generator,validation_data=test_generator,epochs=35,batch_size=len(train_generator))

In [9]:
# Evaluate on test dataset
model.evaluate(test_generator)

6/6 [==============================] - 32s 5s/step - loss: 0.4863 - accuracy: 0.7742 - precision_1: 0.9184


[0.48630258440971375, 0.774193525314331, 0.918367326259613]

In [28]:
# Define a function to make predictions
def defection_predictor(img_path):
    # Load the image from file
    img = image.load_img(img_path, target_size=(224, 224))

    # Convert the image to a numpy array
    img_array = image.img_to_array(img)

    # Expand the dimensions to match the model's expected input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Use the model to predict
    prediction=model.predict(img_array)
    if prediction>=0.65:
       print('Good Tyre.')
    else:
      print('Defective Tyre.')


# Example usage
image_path = "/content/train/defective/Defective (10).jpg"
img_array = defection_predictor(image_path)


1/1 [==============================] - 0s 40ms/step
Defective Tyre


In [ ]:
# Save the model
model.save("model_tyre2.h5")